In [ ]:
# Useful Links
# 1. https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
# 2. https://machinelearningknowledge.ai/keras-implementation-of-resnet-50-architecture-from-scratch/
# 3. https://towardsdatascience.com/resnets-for-cifar-10-e63e900524e0 (Understanding the resnet architecture for cifar10)


In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
PATH = '/content/drive/MyDrive/PhD Studies/fall21/ADVML/results/skip3/'

Mounted at /content/drive/


# RESNET50 WITH SKIP CONNECTION 3: Implemented by Lokesh

In [ ]:
 for i in range(5):
  # Load necessary libraries
  import os
  import pandas as pd
  import numpy as np
  #import cv2
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import backend as K
  from tensorflow.keras.optimizers import SGD, Adam
  #from google.colab.patches import cv2_imshow
  from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau, CSVLogger
  from tensorflow.keras.models import Sequential, Model,load_model
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D,Dropout
  from tensorflow.keras.preprocessing import image
  from tensorflow.keras.initializers import glorot_uniform
  from matplotlib import pyplot as plt
  from tensorflow.keras.utils import to_categorical, plot_model

  def load_dataset(dName="CIFAR10"):
      dataset = None
      num_classes = None
      if dName == "CIFAR10":
          num_classes = 10
          dataset = tf.keras.datasets.cifar10.load_data()
      if dName == "CIFAR100":
          num_classes = 100
          dataset = tf.keras.datasets.cifar100.load_data()
      (X_train, y_train), (X_test, y_test) = dataset
      # Convert target value to categorical values
      # One-hot-encoded target values
      y_train = to_categorical(y_train,num_classes)
      y_test = to_categorical(y_test, num_classes)
      
      return (X_train, y_train),(X_test, y_test)


  def divideDataset(X_train, y_train, X_test,y_test):
      dataLength = X_train.shape[0]
      trainLen=0
      dataTrain = []
      dataTest = []
      percent = 0.2
      while(trainLen<dataLength):
          trainLen = int(X_train.shape[0]*percent)
          print(X_train[:trainLen].shape)
          train = (X_train[:trainLen],y_train[:trainLen])
          retriveLen = int(X_test.shape[0]*percent)
          test = (X_test[:retriveLen],y_test[:retriveLen])
          #print(tuple(train))
          dataTrain.append(train)
          dataTest.append(test)
          print("\n")
          percent +=0.2
      return dataTrain, dataTest

  (X_train, y_train),(X_test, y_test) = load_dataset()
  dataTrain, dataTest = divideDataset(X_train, y_train, X_test, y_test) # this contains the list of 5 different datasets
  def describeDataset(X_train, y_train, X_test, y_test):
      print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
      print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))

  def normalizeInput(X_train,X_test):
      X_train = X_train.astype('float32')
      X_test = X_test.astype('float32')
      mean = np.mean(X_train, axis=(0,1,2,3))
      std = np.std(X_train,axis=(0,1,2,3))
      X_train = (X_train-mean)/(std+1e-7)
      X_test = (X_test-mean)/(std+1e-7)
      
      return X_train, X_test

  # Resnet50 
  def identityBlock(X, f, filters, stage, block, p=0.0):
      k_initializer = glorot_uniform(seed=0)
      conv_name = 'res'+str(stage)+block+'_branch'
      bat_name = 'bn'+str(stage)+block+'_branch'
      
      F1, F2, F3 = filters
      
      X_skip = X
      
      X = Conv2D(filters=F1,kernel_size=(1,1),strides =(1,1),padding='valid',name=conv_name+'2a',
                kernel_initializer=k_initializer)(X)
      X = BatchNormalization(axis=3,name=bat_name+'2a')(X)
      X = Activation('relu')(X)
      X = Dropout(p)(X)
      
      X = Conv2D(filters=F2, kernel_size=(f,f),strides=(1,1),padding='same',name=conv_name+'2b',
                kernel_initializer=k_initializer)(X)
      X = BatchNormalization(axis=3,name=bat_name+'2b')(X)
      X = Activation('relu')(X)
      X = Dropout(p)(X)
      
      X = Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name+'2c',
                kernel_initializer=k_initializer)(X)
      X = BatchNormalization(axis=3,name=bat_name+'2c')(X)
      
      X = Add()([X,X_skip])
      X = Activation('relu')(X)
      X = Dropout(p)(X)
      
      return X

  def convolutionalBlock(X, f, filters, stage, block, s=2,p=0.0):
      k_init = glorot_uniform(seed=0)
      conv_name = 'res'+str(stage)+block+'_branch'
      bat_name = 'bn'+str(stage)+block+'_branch'
      
      F1, F2, F3 = filters
      
      X_skip = X
      
      X = Conv2D(filters=F1,kernel_size=(1,1),strides=(s,s),name=conv_name+'2a',kernel_initializer=k_init)(X)
      X = BatchNormalization(axis=3,name=bat_name+'2a')(X)
      X = Activation('relu')(X)
      X = Dropout(p)(X)
      
      X = Conv2D(filters=F2, kernel_size=(f,f),strides=(1,1),padding='same', name=conv_name+'2b',kernel_initializer=k_init)(X)
      X = BatchNormalization(axis=3,name=bat_name+'2b')(X)
      X = Activation('relu')(X)
      X = Dropout(p)(X)
      
      X = Conv2D(filters=F3, kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name+'2c',kernel_initializer=k_init)(X)
      X = BatchNormalization(axis=3,name=bat_name+'2c')(X)
      
      X_skip = Conv2D(filters=F3,kernel_size=(1,1),strides=(s,s), padding='valid',name=conv_name+'1',
                    kernel_initializer=k_init)(X_skip)
      X_skip = BatchNormalization(axis=3,name=bat_name+'1')(X_skip)
      
      X = Add()([X, X_skip])
      X = Activation('relu')(X)
      X = Dropout(p)(X)
      
      return X
  def ResNet50(input_shape=(64,64,3), classes=10):
      dropoutProb = 0.0
      X_input = Input(shape=(input_shape))
      X = ZeroPadding2D((3,3))(X_input)
      
      
      X = Conv2D(64,kernel_size=(7,7),strides=(2,2),name='conv1',kernel_initializer=glorot_uniform(seed=0))(X)
      X = BatchNormalization(axis=3,name='bn_conv1')(X)
      X = Activation('relu')(X)
      X = MaxPooling2D((3,3),strides=(2,2))(X)
      
      X = convolutionalBlock(X, f=3, filters=[64,64,256],stage=2,block='a',s=1)
      X = identityBlock(X,3,[64,64,256],stage=2,block='b')
      X = identityBlock(X,3,[64,64,256],stage=2,block='c')
      
      
      X = convolutionalBlock(X, f=3, filters=[128,128,512],stage=3,block='a',s=2)
      X = identityBlock(X,3,[128,128,512],stage=3,block='b')
      X = identityBlock(X,3,[128,128,512],stage=3,block='c')
      X = identityBlock(X,3,[128,128,512],stage=3,block='d')
    
      
      X = convolutionalBlock(X, f=3, filters=[256,256,1024],stage=4,block='a',s=2)
      X = identityBlock(X,3,[256,256,1024],stage=4,block='b')
      X = identityBlock(X,3,[256,256,1024],stage=4,block='c')
      X = identityBlock(X,3,[256,256,1024],stage=4,block='d')
      X = identityBlock(X,3,[256,256,1024],stage=4,block='e')
      X = identityBlock(X,3,[256,256,1024],stage=4,block='f')
      
      
      
      X = convolutionalBlock(X, f=3, filters=[512,512,2048],stage=5,block='a',s=2)
      X = identityBlock(X,3,[512,512,2048],stage=5,block='b')
      X = identityBlock(X,3,[512,512,2048],stage=5,block='c')
      
      
      X = AveragePooling2D((2,2),name='avg_pool',padding='same')(X)
      
      X = Flatten()(X)
      X = Dropout(dropoutProb)(X)
      X = Dense(classes, activation='softmax',name='fc'+str(classes),kernel_initializer=glorot_uniform(seed=0))(X)
      
      model = Model(inputs=X_input,outputs=X, name='LResNet50')
      
      return model
  model = ResNet50(input_shape=(32,32,3),classes=10)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  X_train, X_test = normalizeInput(dataTrain[i][0], dataTest[i][0])
  y_train = dataTrain[i][1]
  y_test = dataTest[i][1]
  earlyStop = EarlyStopping(monitor='loss',patience=5,mode='auto')
  csvLogger = CSVLogger("/content/drive/MyDrive/PhD Studies/fall21/ADVML/results/skip3/"+str(i)+".csv")
  history = model.fit(X_train,y_train, epochs=300,batch_size=256,validation_data=(X_test,y_test),verbose=1, callbacks=[csvLogger,earlyStop])